In [9]:
# THIS WORKS WOHOOO

import os
import json
from groq import Groq
import api_keys

groq = Groq(api_key=api_keys.api_key_groq) 

compound = "PTAA"

with open('paper_text.txt', 'r') as f:
    paper_text = f.read()

chat_completion = groq.chat.completions.create(
    messages=[
        {
            "role": "system",
            "content": "You are a solar cell scientist proficient in reading papers. You output only the chemical name of the compound asked for, nothing else.",
            #"content": "You are helpful memory recorder. Write outputs in JSON schema.\n",
            #f" The JSON object must use the schema: {json.dumps(my_schema.model_json_schema(), indent=1)}",
        },
        {
            "role": "user",
            "content": f"What is the chemical name pertaining to this abbreviation: {compound}? You can find it in this text: {paper_text}.",
        }
    ],
    model="llama3-70b-8192",
    #response_format={"type": "json_object"},
)

print(chat_completion.choices[0].message.content)

Poly(triaryl amine)


# Old Llama Code

In [ ]:
import json
from llamaapi import LlamaAPI
import pickle # only necessary for paper reading currently

# Initialize the SDK
llama = LlamaAPI(api_keys.api_key_llama)

compound = "PTAA"
# Load paper text. Necessary because chemdataextractor does not run in nomad.env
# reminder to self: I manually cut the paper_text to be under the token limit
with open('simple_paper_imitation.txt', 'r') as f:
    paper_text = f.read()

# Build the API request
api_request_json = {
    "messages": [
        {"role": "user", "content": f"What is the chemical name pertaining to this abbreviation: {compound}? You can find it in this text: {paper_text}"},
    ],
    "functions": [
        {
            "name": "get_compound_name",
            "description": "Extract the chemical name of a given compound abbreviation from a paper",
            "parameters": {
                "type": "object",
                "properties": {
                    "proper_name": {
                        "type": "string",
                        "description": "The chemical name associated with the abbreviation.",
                    }
                },
            },
            "required": ["proper_name"],
        }
    ],
    "stream": False,
    "function_call": "get_compound_name",
}

# Execute the Request
response = llama.run(api_request_json)
if response.status_code != 200:
    print(f"Request failed with status code {response.status_code}")
else:
    print(json.dumps(response.json(), indent=2))
